In [3]:
import polars as pl

commanders = pl.read_parquet("./../data/processed/commanders.parquet")
commanders.describe()

statistic,name,colorIdentity,colors,faceConvertedManaCost,keywords,loyalty,manaCost,manaValue,power,subtypes,supertypes,toughness,text,type,types,setCodes,page_content
str,str,str,str,f64,str,str,str,f64,str,str,str,str,str,str,str,f64,str
"""count""","""656""","""656""","""656""",35.0,"""461""","""0""","""647""",656.0,"""656""","""656""","""656""","""656""","""655""","""656""","""656""",656.0,"""656"""
"""null_count""","""0""","""0""","""0""",621.0,"""195""","""656""","""9""",0.0,"""0""","""0""","""0""","""0""","""1""","""0""","""0""",0.0,"""0"""
"""mean""",null,null,null,2.657143,null,null,null,3.920732,null,null,null,null,null,null,null,null,null
"""std""",null,null,null,2.057166,null,null,null,1.520284,null,null,null,null,null,null,null,null,null
"""min""","""Abuelo, Ancest…","""""","""""",0.0,"""Adapt""",null,"""{10}""",1.0,"""*""","""Aetherborn, Ro…","""Legendary""","""*""","""({B/P} can be …","""Legendary Crea…","""Creature""",null,"""Legendary Crea…"
"""25%""",null,null,null,1.0,null,null,null,3.0,null,null,null,null,null,null,null,null,null
"""50%""",null,null,null,3.0,null,null,null,4.0,null,null,null,null,null,null,null,null,null
"""75%""",null,null,null,4.0,null,null,null,5.0,null,null,null,null,null,null,null,null,null
"""max""","""Éowyn, Shieldm…","""W""","""W""",7.0,"""Will of the co…",null,"""{X}{W}""",12.0,"""9""","""Zombie, Mutant…","""Legendary""","""9""","""{W/B}{W/B}: If…","""Legendary Crea…","""Creature""",null,"""Legendary Crea…"


In [6]:
commanders_subset = commanders.slice(0, 10)

In [15]:
import langchain
import langchain.globals
import langchain_openai

langchain.globals.set_debug(True)
langchain.globals.set_verbose(True)

llm = langchain_openai.OpenAI()

In [ ]:

from langchain_community.document_loaders import PolarsDataFrameLoader

loader = PolarsDataFrameLoader(commanders_subset)

# Use lazy load for larger table, which won't read the full table into memory
for i in loader.lazy_load():
    print(i)

## Naive classification

In [33]:
from langchain.prompts import PromptTemplate

commander_classification = PromptTemplate.from_template(
    """Classify this commander: {commander_data}"""
)

prompts = []
for i in commanders_subset.rows():
    prompts.append(commander_classification.format(commander_data=i))

responses = []
for i in prompts:
    llm.invoke(i)
    responses.append(llm.invoke(i))

print(responses)

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Classify this commander: ('Abuelo, Ancestral Echo', 'U, W', 'U, W', None, 'Flying, Ward', None, '{1}{W}{U}', 3.0, '2', 'Spirit', 'Legendary', '2', \"Flying, ward {2}\\\\n{1}{W}{U}: Exile another target creature or artifact you control. Return it to the battlefield under its owner's control at the beginning of the next end step.\", 'Legendary Creature — Spirit', 'Creature', ['LCI', 'LCI'], \"Legendary Creature — SpiritFlying, ward {2}\\\\n{1}{W}{U}: Exile another target creature or artifact you control. Return it to the battlefield under its owner's control at the beginning of the next end step.Flying, Ward\")"
  ]
}
[llm/end] [1:llm:OpenAI] [1.82s] Exiting LLM run with output:
{
  "generations": [
    [
      {
        "text": "\nThis commander is a legendary creature with the colors of blue and white. It has a converted mana cost of 3 and a power and toughness of 2. It is classified as a Spirit and has the a

## Few shot classification

In [46]:
example_commanders = commanders.filter(pl.col("name").is_in(["Atraxa, Grand Unifier", "Sauron, the Dark Lord", "The Necrobloom", "Gishath, Sun's Avatar", "Kumena, Tyrant of Orazca"]))

for i in example_commanders.rows():
    print(i)


('Atraxa, Grand Unifier', 'B, G, U, W', 'B, G, U, W', None, 'Deathtouch, Flying, Lifelink, Vigilance', None, '{3}{G}{W}{U}{B}', 7.0, '7', 'Phyrexian, Angel', 'Legendary', '7', 'Flying, vigilance, deathtouch, lifelink\\nWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.)', 'Legendary Creature — Phyrexian Angel', 'Creature', ['ONE', 'ONE', 'ONE', 'ONE'], 'Legendary Creature — Phyrexian AngelFlying, vigilance, deathtouch, lifelink\\nWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, b

In [107]:
def commander_to_prompt_argument(df, commander_name) -> str:
    """Get single row that matches commnader name, then return a simple string with all formatting removed."""
    data = df.filter(pl.col("name") == commander_name).select(
        pl.first("name", "text", "power", "toughness", "manaCost", "type")
    )

    # for each value in each collumn append into a string
    data_str = ""
    for i in data.rows():
        for j in i:
            data_str += str(j) + " "

    # remove all \, \\n and {} from the string
    data_str = data_str.replace("\\", "").replace("\n", "").replace("{", "").replace("}", "")
    return data_str

commander_to_prompt_argument(example_commanders, "Atraxa, Grand Unifier")

'Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel '

In [108]:
from langchain.prompts import FewShotPromptTemplate

examples = [
    {
        "commander": commander_to_prompt_argument(example_commanders, "Atraxa, Grand Unifier"),
        "classification": "Types Matter, Voltron, Enters the battlefield",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders,"Sauron, the Dark Lord"),
        "classification": "Ring Matters, Armies Aggro, Opponent Casts Benefit, Wheel",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders, "The Necrobloom"),
        "classification": "Lands Matter, Token Generation, Dredge",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders,"Gishath, Sun's Avatar"),
        "classification": "Commander Damage, Dinosaur Tribal",
    },
    {
        "commander": commander_to_prompt_argument(example_commanders,"Kumena, Tyrant of Orazca"),
        "classification": "Merfolk Tribal, +1/+1 Counters",
    },
]

example_prompt = PromptTemplate(
    input_variables=["commander", "classification"],
    template="""Classify this commander: {commander}, Classifications: {classification}""",
)

print(example_prompt.format(**examples[0]))

Classify this commander: Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel , Classifications: Types Matter, Voltron, Enters the battlefield


In [113]:
query_commander = commander_to_prompt_argument(
    commanders, "Chiss-Goria, Forge Tyrant"
)

query_commander

'Chiss-Goria, Forge Tyrant Affinity for artifacts (This spell costs 1 less to cast for each artifact you control.)nFlying, hastenWhenever Chiss-Goria, Forge Tyrant attacks, exile the top five cards of your library. You may cast an artifact spell from among them this turn. If you do, it has affinity for artifacts. 5 4 6RRR Legendary Creature — Dragon '

In [114]:
prompt = FewShotPromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
    suffix="Commander: {commander}",
    input_variables=["commander"],
)

print(prompt.format(commander=query_commander))

Classify this commander: Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel , Classifications: Types Matter, Voltron, Enters the battlefield

Classify this commander: Sauron, the Dark Lord Ward—Sacrifice a legendary artifact or legendary creature.nWhenever an opponent casts a spell, amass Orcs 1.nWhenever an Army you control deals combat damage to a player, the Ring tempts you.nWhenever the Ring tempts you, you may discard your hand. If you do, draw four cards. 7 6 3UBR Legendary Creature — Avatar Horror , Classifications: Ring Matters, Armies Aggro, Opponent Casts Benefit, Wheel

C

In [115]:
llm.invoke(prompt.format(commander=query_commander))

[llm/start] [1:llm:OpenAI] Entering LLM run with input:
{
  "prompts": [
    "Classify this commander: Atraxa, Grand Unifier Flying, vigilance, deathtouch, lifelinknWhen Atraxa, Grand Unifier enters the battlefield, reveal the top ten cards of your library. For each card type, you may put a card of that type from among the revealed cards into your hand. Put the rest on the bottom of your library in a random order. (Artifact, battle, creature, enchantment, instant, land, planeswalker, and sorcery are card types.) 7 7 3GWUB Legendary Creature — Phyrexian Angel , Classifications: Types Matter, Voltron, Enters the battlefield\n\nClassify this commander: Sauron, the Dark Lord Ward—Sacrifice a legendary artifact or legendary creature.nWhenever an opponent casts a spell, amass Orcs 1.nWhenever an Army you control deals combat damage to a player, the Ring tempts you.nWhenever the Ring tempts you, you may discard your hand. If you do, draw four cards. 7 6 3UBR Legendary Creature — Avatar Horror

', Classifications: Artifact Affinity, Flying, Card Advantage'